# Загрузка библиотек

In [11]:
# Датасет с вопросами

!git clone https://github.com/vladislavneon/RuBQ.git

In [10]:
# Датасет Quora

!gdown --id 11Q2g2aA-n7fYK95XEB3d6m0CWKSt6rob
!unzip quora_question_pairs_rus.csv.zip

# Импорт библиотек

In [233]:
import fasttext
import requests
import json
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [56]:
import nltk.data
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vaceslavefimov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Загрузка данных

In [50]:
filepath = "RuBQ/RuBQ_2.0/RuBQ_2.0_test.json"
with open(filepath, mode="r", encoding="utf-8") as file:
    lines = file.readlines()
    
questions = [result_json['question_text'] for result_json in json.loads(''.join(lines))]

Всего вопросов: 2330


In [24]:
quora_df = pd.read_csv('quora_question_pairs_rus.csv').drop('Unnamed: 0', axis=1)
quora_df

,question1,question2,is_duplicate
0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1
...,...,...,...
404283,сколько ключевых слов есть на языке программир...,сколько ключевых слов есть в языке программиро...,0
404284,"ты веришь, что есть жизнь после смерти","правда ли, что есть жизнь после смерти",1
404285,что такое одна монета,что это за монета,0
404286,какова приблизительная годовая стоимость жизни...,"У меня проблема с волосами, но я хочу использо...",0


# Обработка данных

In [196]:
def text_to_wordlist(text, remove_stopwords=False):
    text = re.sub(r"http[s]?://(?:[а-яА-Я]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    text = re.sub("[^а-яА-Я]"," ", text)
    words = text.lower().split()
    if remove_stopwords:
        stops = stopwords.words('russian')
        words = [w for w in words if not w in stops]
    return (words)


def process_text(text, tokenizer, remove_stopwords=False):
    raw_texts = tokenizer.tokenize(text.strip())
    texts = []
    for raw_text in raw_texts:
        if len(raw_text) > 0:
            texts.append(text_to_wordlist(raw_text, remove_stopwords))
    return texts

In [159]:
processed_texts = []
tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')

texts = questions + quora_df.question1.tolist() + quora_df.question2.tolist()
texts = list(filter(lambda x: pd.notnull(x), texts))
for text in texts:
    processed_text = process_text(text, tokenizer)
    processed_texts += processed_text

Обработано 100000 текстов
Обработано 200000 текстов
Обработано 300000 текстов
Обработано 400000 текстов
Обработано 500000 текстов
Обработано 600000 текстов
Обработано 700000 текстов
Обработано 800000 текстов


In [160]:
filename = 'processed_texts.txt'

with open(filename, 'w') as f:
    for text in processed_texts:
        f.write(' '.join(text))
        f.write('\n')

# Обучение модели

In [161]:
fasttext_model = fasttext.train_unsupervised(filename, minn=3, maxn=4, dim=300) 

In [175]:
# построим эмбеддинг для каждого вопроса в датасете, удаляя при этом дубликаты

texts_df = pd.DataFrame(data={
    'text': texts,
    'embedding': [fasttext_model.get_sentence_vector(text) for text in texts]
}).drop_duplicates(subset=['text']).reset_index(drop=True)
texts_df

,text,embedding
0,Что может вызвать цунами?,"[-0.026511569, 0.0022450201, -0.054608483, -0...."
1,Кто написал роман «Хижина дяди Тома»?,"[-0.043679472, 0.063685976, 0.008840534, -0.00..."
2,Кто автор пьесы «Ромео и Джульетта»?,"[0.017778367, 0.020919215, 0.010113756, 0.0297..."
3,Как называется столица Румынии?,"[0.023830201, 0.02008646, -0.0018236416, -0.03..."
4,На каком инструменте играл Джимми Хендрикс?,"[-0.013115338, 0.030028015, 0.012138782, 0.015..."
...,...,...
532059,что будет означать обновление процессора до 20...,"[-0.024191316, 0.068901695, -0.039634127, 0.01..."
532060,что джайнизм говорит о геях и гомосексуализме,"[-0.0003970106, 0.042277154, -0.038078535, -0...."
532061,что это за монета,"[0.00208145, 0.042230263, -0.04349307, -0.0177..."
532062,"У меня проблема с волосами, но я хочу использо...","[0.009293586, 0.06122627, 0.004851475, -0.0065..."


In [165]:
def cosine(u, v):
    res = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    return res

In [265]:
def check_spelling(text):
    domain = "https://speller.yandex.net/services/spellservice.json"
    words = text.split()
    if len(words) == 1:
        request = requests.get(domain + "/checkText?text=" + words[0])
        if request.json():
            return request.json()[0]["word"], request.json()[0]["s"]
        else:
            return None
    
    elif len(words) > 1:
        words = "+".join(words)
        request = requests.get(domain + "/checkText?text=" + words)
        if request.json():
            response = [(i["word"], i["s"]) for i in request.json()]
            return response
        else:
            return None
    return None

In [291]:
def most_similar_texts(text, n=10):
    
    edited_words = []
    words = text.split()
    for word in words:
        edited_word = check_spelling(word)
        edited_words.append(word if edited_word is None else edited_word[1][0])
    edited_text = ' '.join(edited_words)
    print(f'Исправленный текст: {edited_text}')
    
    embedded_text = fasttext_model.get_sentence_vector(edited_text)
    cosines = [cosine(x, embedded_text) for x in texts_df['embedding']]
    argpartition = np.argpartition(cosines, -n)[-n:]
    
    return pd.DataFrame(data={
        'text': [texts_df.loc[i, 'text'] for i in argpartition],
        'cosine': [cosines[i] for i in argpartition]
    }).sort_values(by='cosine', ascending=False).reset_index(drop=True)

# Вывод наиболее похожих вопросов

In [292]:
text = 'Какой город самый большой в мире?'
most_similar_texts(text)

Исправленный текст: Какой город самый большой в мире?


,text,cosine
0,какой самый большой город в мире,0.955754
1,какой самый большой город в мире по площади,0.903134
2,какой самый большой океан в мире,0.901759
3,какой самый большой город в Финляндии?,0.898107
4,какой самый большой город,0.875569
5,самый большой мошенник в мире,0.869127
6,самый красивый город в мире,0.859069
7,самый большой хит-фильм в мире,0.858029
8,почему город Нью-Йорк - самый большой город в мире,0.857540
9,самый большой грузовик в мире,0.853889


In [295]:
text = 'Какой вуз самый лучший?'
most_similar_texts(text)

Исправленный текст: Какой вуз самый лучший?


,text,cosine
0,какой самый быстрый способ стать лучше,0.822894
1,самый лучший самый крутой кошелек,0.818989
2,Какой лучший онлайн-институт,0.818525
3,какой лучший онлайн-университет,0.818102
4,какой самый быстрый способ стать ведьмой,0.812313
5,какой самый быстрый способ обучения,0.810570
6,какой город самый лучший в Индии,0.809875
7,какой самый быстрый способ учиться,0.809622
8,какой лучший и самый простой способ снять стресс,0.809098
9,Какой лучший способ выбрать университет?,0.802177


In [294]:
text = 'Какой драматург написал пьесу «Ромео и Джульетта»'
most_similar_texts(text)

Исправленный текст: Какой драматург написал пьесу «Ромео и Джульетта»


,text,cosine
0,Кто автор пьесы «Ромео и Джульетта»?,0.833784
1,Какую оперу на сюжет «Ромео и Джульетты» написал композитор Винченцо Беллини?,0.788281
2,какой ваш любимый тип музыки хип-хоп и рэп б хип-хоп и рок с твердым металлом и скалой,0.753673
3,"композитор камерной музыки, который написал пьесу для родной американской флейты, сказал, что все на флейтисты звучат одинаково и не являются настоящими музыкантами, это правда",0.741522
4,кто-нибудь может написать лирику и перевести песню lekha из драмы танца кучипуди бхама калапам,0.741512
5,"Какой композитор написал музыку к кинофильмам «Семнадцать мгновений весны» и «Ирония судьбы, или С лёгким паром!»?",0.737834
6,который поет песню из золотого яйца в Гарри Поттере и бокал огня,0.721192
7,"Кто является режиссером и комедии «Дети Дон Кихота», и военной драмы «Служили два товарища», и приключенческого фильма «Два капитана»?",0.716512
8,который написал рамаяну и махабхарату,0.715487
9,который написал махабхарату и рамаяну,0.715487


In [296]:
text = 'Какие причины у глобального потепления?'
most_similar_texts(text)

Исправленный текст: Какие причины у глобального потепления?


,text,cosine
0,каковы причины глобального потепления,0.855691
1,каковы основные причины глобального потепления,0.827622
2,какие последствия глобального потепления,0.818347
3,какие страны выиграют от глобального потепления,0.783171
4,каковы экологические последствия глобального потепления,0.777425
5,каковы некоторые важные причины и следствия в отношении глобального потепления,0.768960
6,есть ли у нас хорошие способы остановить глобальное потепление,0.756727
7,какие возможные выгоды могут быть для глобального потепления,0.755660
8,каковы наиболее вероятные последствия глобального потепления,0.753545
9,каковы потенциальные социальные последствия глобального потепления,0.753325


In [297]:
text = 'Какой сериал можно посмотреть на выходных?'
most_similar_texts(text)

Исправленный текст: Какой сериал можно посмотреть на выходных?


,text,cosine
0,"где я могу посмотреть драконский бал супер английский, дублированный на телевизоре",0.750216
1,"Бэтмен: анимированная серия не на какой-либо сети кабельного телевидения или netflix, почему так трудно найти, где я могу законно ее посмотреть",0.748812
2,"что такое ужасный фильм, который можно посмотреть на Хэллоуин",0.745502
3,можно посмотреть розовый фильм с семьей,0.738775
4,какой фильм я должен смотреть на netflix,0.732154
5,"что такое классный бесплатный сайт, на котором можно смотреть документальные фильмы полной длины",0.731401
6,на каком веб-сайте я могу смотреть бесплатные сериалы,0.730397
7,что стоит посмотреть на хинди короткометражные фильмы на youtube,0.726977
8,"как мне посмотреть, кто подписался на меня на youtube",0.723905
9,"сколько нравится на facebook, вам нужно зарабатывать деньги, то же самое можно посмотреть на просмотрах youtube",0.722869


In [298]:
text = 'Кокой город самй бальшой?'
most_similar_texts(text)

Исправленный текст: Какой город самый большой


,text,cosine
0,какой самый большой город,0.948476
1,какой самый большой город в мире,0.863520
2,какой самый большой город в Финляндии?,0.856085
3,какой самый большой секрет,0.844138
4,какой парень самый большой поворот,0.821552
5,Какой океан самый маленький?,0.817229
6,какой самый большой секрет Индии,0.813510
7,какой самый большой океан в мире,0.798750
8,какой самый большой город в мире по площади,0.796806
9,какой самый красивый город в Новой Англии,0.793942


In [304]:
text = 'Как можно научыться еграть в шашматы?'
most_similar_texts(text)

Исправленный текст: Как можно научиться играть в шахматы


,text,cosine
0,как можно научиться играть в шахматы?,0.941544
1,как научиться играть в шахматы,0.910208
2,как научиться играть в шахматы?,0.904903
3,как я могу научиться играть в шахматы,0.870231
4,почему я должен учиться играть в шахматы,0.863234
5,как играть в шахматы,0.858707
6,как я научиться играть в футбол,0.849203
7,как вы научились играть в шахматы,0.845436
8,как я могу научиться играть в шахматы в своем уме,0.844303
9,как не играть в шахматы,0.840227


In [306]:
text = 'Как выучитъ ангийций язик?'
most_similar_texts(text)

Исправленный текст: Как выучить английский язык


,text,cosine
0,как трудно английскому говорящему выучить японский язык,0.840291
1,как можно хорошо выучить английский язык,0.834835
2,официальный английский язык,0.831228
3,английский язык - самый латинизированный немецкий язык,0.822856
4,как улучшить английский язык,0.822718
5,почему языку родного английского языка сложнее изучать китайский язык,0.821118
6,английский язык - это рабский язык,0.818943
7,как носители языка изучают английский язык,0.814217
8,немецкий язык лучший язык,0.803009
9,как возник английский язык,0.802773


In [310]:
text = 'Кок я магу зстатъ сомым умным?'
most_similar_texts(text)

Исправленный текст: Кок я магу зстатъ самым умным?


,text,cosine
0,как я могу стать очень умным,0.773474
1,как я могу стать умным и умным,0.763547
2,"я слабый в математике, я могу стать очень хорошим в этом",0.758426
3,как я должен стать умным,0.754234
4,как я могу стать умным,0.754032
5,"как я могу стать популярным на кворе, не будучи умным",0.753904
6,"мне 19 лет, и я хотел знать, как я могу стать очень умным без образования в колледже, какие инструменты мне нужны или ресурсы",0.746496
7,"мне 20 лет, и я хотел знать, как я могу стать очень умным без образования в колледже, какие инструменты мне нужны или ресурсы",0.746348
8,"я плохо разбираюсь в математике, я провалил базовую алгебру колледжа четыре раза, прежде чем я едва скрестил, как я могу стать лучше, поэтому я могу переключить свой майор на инженерную",0.742045
9,как я могу стать хорошим лидером для умных,0.741984


In [311]:
text = 'Что нжно сделодь, чтобы вставтъ утром?'
most_similar_texts(text)

Исправленный текст: Что нужно сделать чтобы вставать утром?


,text,cosine
0,"что я должен сделать, чтобы встать рано утром",0.862036
1,"что я должен сделать, чтобы проснуться рано утром?",0.847433
2,"что нужно сделать, чтобы сделать кого-то счастливым",0.826162
3,"что я могу сделать, чтобы встать рано",0.824727
4,"что я могу сделать, чтобы быстрее проснуться утром",0.819368
5,"что нужно сделать, чтобы сделать правду или посмеяться",0.817738
6,"это лучшее упражнение, которое нужно сделать утром",0.816254
7,"что нужно сделать, чтобы похудеть",0.813277
8,"что вам нужно сделать, чтобы не спать 2 дня подряд",0.811270
9,"что я должен сделать, чтобы почувствовать свежесть утром",0.811147


In [320]:
text = 'Как нойти в себе уввереност?'
most_similar_texts(text)

Исправленный текст: Как найти в себе уверенность


,text,cosine
0,как повысить уверенность в себе,0.841364
1,как я могу вернуть себе уверенность в себе,0.830449
2,как восстановить свою уверенность в себе,0.823380
3,как я развиваю уверенность в себе,0.810765
4,каковы способы вернуть себе уверенность в себе,0.809136
5,как я могу обрести уверенность в себе,0.807449
6,"я теряю уверенность в себе и мотивацию, как я могу построить свою уверенность в себе и мотивацию",0.801707
7,как я могу вернуть утраченную уверенность в себе,0.801561
8,как я могу укрепить уверенность в себе,0.801338
9,как я могу повысить уверенность в себе,0.801252
